## r0 to r1 calibrator


In [ ]:

%load_ext autoreload
%autoreload 2

from ctapipe.io import event_source
import sys
 
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import sys
from scipy.stats import norm
from traitlets.config.loader import Config 
from ctapipe import utils
from ctapipe_io_lst import LSTEventSource


# ctapipe modules
from ctapipe.visualization import CameraDisplay
from ctapipe.image.extractor import *
from ctapipe.containers import PedestalContainer
from ctapipe.io.hdf5tableio import HDF5TableWriter, HDF5TableReader
from lstchain.calib.camera.r0 import LSTR0Corrections
from lstchain.calib.camera.calibrator import LSTCameraCalibrator


tel_id=1 # this is the tel_id for LST-1


In [ ]:
r0_r1_calibrator = LSTR0Corrections(
    pedestal_path="../../data/calibration/drs4_pedestal.Run1623.0000.fits",
    r1_sample_start=2,r1_sample_end=38, tel_id=tel_id)

# r1 to dl1 calibrator
inp="../../data/calibration/calibration.Run1625.0000.hdf5"
time_calib = "../../data/calibration/time_calibration.Run1625.0000.hdf5"
charge_config = Config({
    "LocalPeakWindowSum": {
        "window_shift": 4,
        "window_width": 8 
    }
})


reader = LSTEventSource(input_url="../../data/real_data/LST-1.1.Run01627.0001.fits.fz",
                        max_events=1000)
subarray = reader.subarray

r1_dl1_calibrator = LSTCameraCalibrator(calibration_path = inp,
                               time_calibration_path = time_calib,
                               gain_threshold = 3000,
                               image_extractor="LocalPeakWindowSum",
                               config=charge_config,
                               subarray = subarray
                                )

# shower events

print(f"\n Read {len(reader.multi_file)} total events  in files\n")
print(f"{reader.file_list} ")

In [ ]:
# function to plot charge and time

def plot_charge_time():
    
    dim = event.dl1.tel[tel_id].image.ndim
    #dim = event.dl1.tel[tel_id].image.ndim
    if dim == 1:
        fig = plt.figure(figsize=(16, 8))
        pad = 120
        mask= np.where((event.lst.tel[tel_id].evt.pixel_status >> 2 & 1>0))
    else:
        fig = plt.figure(figsize=(16, 16))
        pad = 220
        mask =[]


    for chan in np.arange(dim):
        if dim == 1:
            image = event.dl1.tel[tel_id].image
        else:   
            image = event.dl1.tel[tel_id].image[chan]

        pad+=1
        ax = plt.subplot(pad)
        plt.tight_layout()
        disp = CameraDisplay(subarray.tels[tel_id].camera.geometry)
        disp.image = image  
        disp.cmap = plt.cm.coolwarm
        disp.set_limits_minmax(0,200)
        disp.axes.text(1.8, 0, f'charge (pe)', rotation=90)
        disp.add_colorbar()
        plt.tight_layout()
        print(f"charge : mean {image.mean()} +- {image.std()}")
        disp.highlight_pixels(mask,linewidth=2)

        pad+=1  
        if dim == 1:
            image = event.dl1.tel[tel_id].peak_time 
        else:   
            image = event.dl1.tel[tel_id].peak_time[chan]

        ax = plt.subplot(pad)  
        plt.tight_layout()
        disp = CameraDisplay(subarray.tels[tel_id].camera.geometry)
        disp.image = image 
        disp.highlight_pixels(mask,linewidth=2)
        disp.set_limits_minmax(-10,10)
        disp.cmap = plt.cm.coolwarm
        disp.axes.text(1.8, 0, f' time (ns)', rotation=90)
        disp.add_colorbar()
        plt.tight_layout()
        print(f"time: mean {image.mean()} +- {image.std()}")


In [ ]:
LSTCameraCalibrator?

In [ ]:
# loop on events
r1_dl1_calibrator = LSTCameraCalibrator(calibration_path = inp,
                               time_calibration_path = time_calib,
                               gain_threshold = None,
                               image_extractor="LocalPeakWindowSum",
                               config=charge_config,
                               subarray=subarray 
                                    )

for i, event in enumerate(reader):
    
    if event.index.event_id%500 == 0:
        print(event.index.event_id)
              
    # calibrate r0 --> r1        
    r0_r1_calibrator.calibrate(event)
   
    # if triggerd event
    if event.r0.tel[tel_id].trigger_type != 32:                   
        # calibrate r1 --> dl1
        r1_dl1_calibrator(event)
        image = event.dl1.tel[tel_id].image
        
        # select image with bright event and stop
        if np.max(image>200):             
            print(f"max pe {np.max(image)}")
            break
            
            
print(f"read event id: {event.index.event_id}, trigger {event.r0.tel[tel_id].trigger_type}")

# plot selected event
plot_charge_time()

In [ ]:
!ls $time_calib

## with gain selection

In [ ]:
# loop on events
r1_dl1_calibrator = LSTCameraCalibrator(calibration_path = inp,
                               time_calibration_path = time_calib,
                               gain_threshold = 3000,
                               image_extractor="LocalPeakWindowSum",
                               config=charge_config,
                               subarray=subarray                                        
                                       )

for i, event in enumerate(reader):
    
    if event.index.event_id%500 == 0:
        print(event.index.event_id)
              
    # calibrate r0 --> r1        
    r0_r1_calibrator.calibrate(event)
   
    # if triggerd event
    if event.r0.tel[tel_id].trigger_type != 32:                   
        
        # calibrate r1 --> dl1
        r1_dl1_calibrator(event)
        image = event.dl1.tel[tel_id].image
        # bit 2 == LG 
        mask = event.lst.tel[tel_id].evt.pixel_status >> 2 & 1
        # select image with bright event and stop
        if(np.max(image>200) and np.max(mask)>0):             
            print(f"max pe {np.max(image)}")
            break
            
            
print(f"read event id: {event.index.event_id}, trigger {event.r0.tel[tel_id].trigger_type}")

# plot selected event
plot_charge_time()

In [ ]:
channel=['HG','LG']

a=np.where(event.dl1.tel[1].image>100)
pix=a[0][0]

fig = plt.figure(300,figsize=(12, 12))
label="after calibration"
label1="before calibration"
chan=0
plt.subplot(2, 2, 1)
plt.title(f"pixel {pix}, channel {channel[chan]}")
plt.plot(event.r1.tel[1].waveform[chan,pix], label=label1)
plt.legend()
plt.subplot(2, 2, 3)
plt.title(f"pixel {pix}, channel {channel[chan]}")
plt.plot(event.dl0.tel[1].waveform[chan,pix], label=label)   
plt.title(f"pixel {pix}, channel {channel[chan]}")
plt.legend()
chan=1
plt.subplot(2, 2, 2)
plt.title(f"pixel {pix}, channel {channel[chan]}")
plt.plot(event.r1.tel[1].waveform[chan,pix], label=label1)
plt.legend()
plt.subplot(2, 2, 4)
plt.plot(event.dl0.tel[1].waveform[chan,pix], label=label)
plt.title(f"pixel {pix}, channel {channel[chan]}")
plt.legend()
